In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from matplotlib import pyplot as plt
from itertools import chain, combinations

In [9]:
raw_data = pd.read_excel('main.xlsx')
required_columns = {'Team Display Name', 'Date', 'Opponent', 'PTS', 'FGM', 'FGA', 
                        'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'TO', 'Win'}
                        
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

team_stats = raw_data.groupby(['Team Display Name']).agg(
        PTS=('PTS', 'sum'),
        FGM=('FGM', 'sum'),
        FGA=('FGA', 'sum'),
        FTM=('FTM', 'sum'),
        FTA=('FTA', 'sum'),
        REB=('REB', 'sum'),
        AST=('AST', 'sum'),
        STL=('STL', 'sum'),
        BLK=('BLK', 'sum'),
        TO=('TO', 'sum'),
        Win=('Win', 'first')
    ).reset_index()

df = team_stats.drop('Team Display Name', axis=1)
df

,PTS,FGM,FGA,FTM,FTA,REB,AST,STL,BLK,TO,Win
0,53,19,57,14,20,29,9,11,5,10,True
1,163,56,162,34,50,99,33,29,2,51,True
2,82,30,64,9,13,30,20,13,5,9,True
3,78,31,66,5,9,49,19,13,10,13,True
4,824,285,775,166,247,465,149,109,37,222,True
...,...,...,...,...,...,...,...,...,...,...,...
174,92,33,77,19,29,43,19,5,2,14,True
175,45,16,41,13,22,24,10,9,3,27,False
176,160,65,129,19,28,70,48,24,18,38,True
177,588,217,547,104,167,314,126,82,28,126,True


In [46]:
def powerset(iterable):
    "Subsequences of the iterable from shortest to longest."
    # powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

subsets = list(powerset(df.columns))
subsets.pop(0)
subsets

[('PTS',),
 ('FGM',),
 ('FGA',),
 ('FTM',),
 ('FTA',),
 ('REB',),
 ('AST',),
 ('STL',),
 ('BLK',),
 ('TO',),
 ('Win',),
 ('PTS', 'FGM'),
 ('PTS', 'FGA'),
 ('PTS', 'FTM'),
 ('PTS', 'FTA'),
 ('PTS', 'REB'),
 ('PTS', 'AST'),
 ('PTS', 'STL'),
 ('PTS', 'BLK'),
 ('PTS', 'TO'),
 ('PTS', 'Win'),
 ('FGM', 'FGA'),
 ('FGM', 'FTM'),
 ('FGM', 'FTA'),
 ('FGM', 'REB'),
 ('FGM', 'AST'),
 ('FGM', 'STL'),
 ('FGM', 'BLK'),
 ('FGM', 'TO'),
 ('FGM', 'Win'),
 ('FGA', 'FTM'),
 ('FGA', 'FTA'),
 ('FGA', 'REB'),
 ('FGA', 'AST'),
 ('FGA', 'STL'),
 ('FGA', 'BLK'),
 ('FGA', 'TO'),
 ('FGA', 'Win'),
 ('FTM', 'FTA'),
 ('FTM', 'REB'),
 ('FTM', 'AST'),
 ('FTM', 'STL'),
 ('FTM', 'BLK'),
 ('FTM', 'TO'),
 ('FTM', 'Win'),
 ('FTA', 'REB'),
 ('FTA', 'AST'),
 ('FTA', 'STL'),
 ('FTA', 'BLK'),
 ('FTA', 'TO'),
 ('FTA', 'Win'),
 ('REB', 'AST'),
 ('REB', 'STL'),
 ('REB', 'BLK'),
 ('REB', 'TO'),
 ('REB', 'Win'),
 ('AST', 'STL'),
 ('AST', 'BLK'),
 ('AST', 'TO'),
 ('AST', 'Win'),
 ('STL', 'BLK'),
 ('STL', 'TO'),
 ('STL', 'Win'),
 ('B

In [51]:
scores = {}
for subset in subsets:
    subset_data = df[list(subset)]
    clusters = SpectralClustering(n_clusters=3, assign_labels='cluster_qr').fit(subset_data)
    labels = clusters.labels_
    scores[subset] = silhouette_score(df, labels)

c:\Users\Air\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\Air\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\Air\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:455: UserWarning: Exited at iteration 462 with accuracies 
[5.09941573e-15 2.56848559e-06 9.43178902e-06 2.64891637e-06]
not reaching the requested tolerance 2.6673078536987305e-06.
Use iteration 404 instead with accuracy 
1.5337506780874247e-06.

  _, diffusion_map = lobpcg(
c:\Users\Air\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:455: UserWarning: Exited postprocessing with accuracies 
[1

In [73]:
{k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}

{('BLK', 'Win'): np.float64(0.7136990338844537),
 ('BLK',): np.float64(0.6879988661837046),
 ('STL', 'Win'): np.float64(0.6500343669148315),
 ('STL',): np.float64(0.5267016327613727),
 ('AST',): np.float64(0.43983693192172224),
 ('AST', 'Win'): np.float64(0.41292907885695784),
 ('FTA', 'Win'): np.float64(0.3876012927495472),
 ('TO', 'Win'): np.float64(0.35498192333995987),
 ('BLK', 'TO'): np.float64(0.3379947341081845),
 ('FTM', 'BLK'): np.float64(0.30025989071431),
 ('TO',): np.float64(0.29191741929108267),
 ('AST', 'BLK', 'Win'): np.float64(0.28135262560897917),
 ('AST', 'STL'): np.float64(0.2674723836797298),
 ('FTM', 'Win'): np.float64(0.22669058932636013),
 ('FTA', 'BLK', 'Win'): np.float64(0.22091137508240197),
 ('FGM', 'Win'): np.float64(0.2158665290045266),
 ('STL', 'BLK', 'Win'): np.float64(0.21331949305556813),
 ('FGM',): np.float64(0.19513452044125132),
 ('AST', 'STL', 'Win'): np.float64(0.18377395608217764),
 ('AST', 'BLK'): np.float64(0.177343985481909),
 ('STL', 'BLK'): n